**Partie II**

In this section, we focus on two other important aspects of deep learning: speeding up learning with GPU cards and the ability to use pretrained networks.

To illustrate the first aspect, we will use the GPUs available under Google Colab. To do this, before starting to read the notebook, go to **Modifier**/**Modifier les param du notebook** and select a GPU.

For the second aspect, we will work on a binary classification problem using a small dataset ("hymenoptera_data"). We will see the benefits of using a network that has already been trained on a larger dataset and a more general classification task.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

In [ ]:
# Check GPU availability
if torch.cuda.is_available():
  device = torch.device("cuda:0") # 0 is the index of the GPU
  print("You are on GPU !")
else:
  print('Change the runtime type to GPU')

## **A.** Load and viz the Hymenoptera dataset:

In [ ]:
# download the dataset
! wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
! unzip -qq hymenoptera_data.zip

In [ ]:
dir_data = 'hymenoptera_data'
print(os.listdir(dir_data))

The dataset is in a standard format, and we can manipulate it with a ready-to-use dataset object of the datasets.ImageFolder class:

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(dir_data, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

print('Dataset sizs:' )
print(dataset_sizes)

Since the provided dataset is very small, we need to maximize its utility. We will produce new images through additional transformations that preserve the nature of the object (data augmentation). \\
In the code, transforms.*RandomResizedCrop()*, *transforms.RandomHorizontalFlip()* and *transforms.RandomVerticalFlip()* apply horizontal or vertical axis symmetry with a probability of 1/2. Note that these transformations might not be suitable for other datasets like MNIST since the mirror image of a digit is generally not another digit. Therefore, transformations often need to be adapted to the nature of the dataset. \\
Some images are presented below.


In [ ]:
def imshow(inp, ax=None, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    if ax is None:
      plt.imshow(inp)
      plt.title(title)
    else:
      ax.imshow(inp)
      ax.set_title(title)

In [ ]:
def plot_batch(images, labels, class_names):
    num_images = len(images)
    fig, axs = plt.subplots(1, num_images, figsize=(15, 5))

    for i in range(num_images):
        axs[i].axis('off')
        imshow(images[i],axs[i],class_names[labels[i]])
    plt.show()
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))
class_names = image_datasets['train'].classes
# Assuming `inputs` is a batch of images and `classes` are the corresponding class labels
plot_batch(inputs, classes, class_names)


## **B.** Using a Graphics Card:

Let's start by checking if, for typical architectures like ResNet, using a GPU significantly improves computation time. We will begin by loading the lightest of the ResNet architectures and select an appropriate loss function.

**Exercise**

- Load an untrained ResNet18. How many total weights does it contain? Check [here](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html).

- How many neurons does the last layer of the network have?

- Modify the last layer of the classifier so that it has as many neurons as there are classes in hymenoptera_data:

In [ ]:
model = models.resnet18(pretrained=False)
# YOUR CODE HERE
raise NotImplementedError()

**Note:** Batch normalization operations facilitate learning but won't be detailed here. In summary, they center/reduce each feature map over the batch and then renormalize. The two renormalization parameters (mean and standard deviation) are trainable.

In [ ]:
# Neurons in the last layer
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Modification of the last layer of the classifier
def get_model(pretrained):
  model = models.resnet18(pretrained=pretrained)
  # YOUR CODE HERE
  raise NotImplementedError()
  return model

get_model(False)

**Exercise:** Is there a softmax layer at the end of the ResNet architecture?

YOUR ANSWER HERE

Now, let's define the negative log-likelihood as the cost function. To compute the log-likelihood, we could add a LogSoftmax layer to the ResNet. Yet it's easier and more common to use a loss function that includes *LogSoftmax*. In this regard, in PyTorch,  *nn.CrossEntropyLoss* combines both *LogSoftmax* and *NLLLoss*.

In [ ]:
loss_fn =  nn.CrossEntropyLoss()

Finally, let's define a function that incorporates the training loop.




**Exercise:**
- Complete the *train_model* function, which takes a model, a loss function, an optimizer, and a number of epochs as arguments.
- Train the ResNet over one epoch with mini-batches of 64 images.

In [ ]:
def train_model(model, loss_fn, optimizer, num_epochs=1):
    since = time.time()

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                # Weights are not updated during the validation phase

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                # 1. zeroing grads
                # YOUR CODE HERE
                raise NotImplementedError()
                with torch.set_grad_enabled(phase == 'train'):
                    # forward step (outputs = ...)
                    # get the predicted classes (preds = ...)
                    # loss computing (loss = ...)
                    # YOUR CODE HERE
                    raise NotImplementedError()
                    if phase == 'train':
                        # backward step
                        # update wieghts
                        # YOUR CODE HERE
                        raise NotImplementedError()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {100*epoch_acc:.2f}%')

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [ ]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=2)
              for x in ['train', 'val']}

model = get_model(pretrained=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Launch the training over 1 epoch:
# model = ...
# YOUR CODE HERE
raise NotImplementedError()

With more than 10 million parameters, training a ResNet on a CPU is much slower than the networks in Part I. \\
Let's repeat the same training using the GPU.

In [ ]:
print(f'Runtime device :{device}')

# Load the model to the GPU:
model = model.to(device)

To load a torch.tensor on GPU, the syntax is the same:

In [ ]:
x = torch.rand(2,1,4,4)
print("On CPU :\n",x)
x = x.to(device)
# Note: You can also use .cuda() without specifying the device name
# but this method is not recommended especially in a multi-gpu environment
print("On GPU :\n",x)

# bring back the x tensor to the CPU RAM:
x = x.to('cpu') # or x.cpu()
print('Back to CPU:\n',x)

**Exercice :**
- Complete the fonction *train_model_gpu* to train the model on GPU.
- Compare the CPU and GPU training times.

In [ ]:
def train_model_gpu(model, loss_fn, optimizer, num_epochs=1):
    # Record the starting time
    since = time.time()
    # Loop through epochs
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Iterate through training and validation phases
        for phase in ['train', 'val']:
            # Set the model to training mode during the training phase, and evaluation mode during validation
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # Initialize counters for loss and correct predictions
            running_loss = 0.0
            running_corrects = 0

            # Iterate through batches in the data loader
            for inputs, labels in dataloaders[phase]:
                # Move inputs and labels to the specified device (GPU)
                # YOUR CODE HERE
                raise NotImplementedError()

                # Zero the gradients in the optimizer (same as in train_model())
                # YOUR CODE HERE
                raise NotImplementedError()
                # Forward pass: compute model outputs and predictions (same as in train_model())
                with torch.set_grad_enabled(phase == 'train'):
                    # YOUR CODE HERE
                    raise NotImplementedError()
                    # Backward pass and optimization step if in the training phase (same as in train_model())
                    # YOUR CODE HERE
                    raise NotImplementedError()
                # Update counters
                running_loss += loss.item() * inputs.size(0)
                acc = torch.sum(preds == labels.data)
                # The 'acc' tensor is distributed across different parts of the GPU
                # Gather the 'acc' tensor on the CPU before accumulation
                # running_corrects += ...
                # YOUR CODE HERE
                raise NotImplementedError()

            # Calculate average loss and accuracy for the epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # Print epoch statistics
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {100*epoch_acc:.2f}%')

    # Calculate and print the total training time
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    # Return the trained model
    return model


model = get_model(pretrained=False).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

model = train_model_gpu(model, loss_fn, optimizer, num_epochs=1)

### **GPU**
A GPU, or Graphics Processing Unit, is a specialized electronic circuit designed to accelerate the processing of images and videos. Originally developed for rendering graphics in video games, GPUs have evolved into highly parallel processors that excel at performing many simultaneous calculations. Unlike CPUs (Central Processing Unit), the GPUs are optimized for parallel processing tasks.

Key characteristics of GPUs include:

1. **Parallel Processing:** GPUs consist of a large number of cores, allowing them to handle multiple tasks simultaneously. This makes them well-suited for parallelizable computations.

2. **Vectorized Processing:** GPUs are designed to efficiently perform operations on vectors and matrices. This is particularly beneficial for tasks such as linear algebra operations, which are common in machine learning and scientific computing.

3. **High Memory Bandwidth:** GPUs have high-speed memory to quickly access and manipulate large amounts of data, making them suitable for tasks that involve substantial data throughput.

4. **Floating-Point Performance:** GPUs are optimized for floating-point arithmetic, which is important for applications that require precision in numerical calculations.

5. **Specialized Compute APIs:** Modern GPUs support specialized compute APIs (Application Programming Interfaces) such as CUDA (Compute Unified Device Architecture) for NVIDIA GPUs or OpenCL (Open Computing Language), which allows developers to offload parallelizable tasks to the GPU.

6. **General-Purpose GPU Computing:** Beyond graphics rendering, GPUs are now widely used for general-purpose computing, including scientific simulations, machine learning, data analysis, and other computationally intensive tasks.

In the context of machine learning, deep learning frameworks like TensorFlow and PyTorch leverage GPUs to accelerate the training of neural networks. The parallel processing capabilities of GPUs enable the simultaneous computation of numerous mathematical operations, significantly speeding up training times compared to using only a CPU.

## **C.**  Improving the Training Procedure:

Before comparing different transfer approaches, let's improve the training procedure a bit. We can add momentum to the gradient descent. The  [increment calculation](https://pytorch.org/docs/master/optim.html#torch.optim.SGD) will depend not only on the current gradient but also on past increment values stored in $d_i$:

\begin{equation}
d_i^{t+1} = momentum \times d_i^{t} +    \dfrac{\partial \mathcal{L_{batch}^{t+1}}} {\partial{\omega_i}} \\
w_i^{t+1}  = w_i^{t} - lr \times d_i^{t+1}
\end{equation}

Here is an example:

`optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)`


Next, we'll add a *scheduler* that gradually decreases the learning rate *lr*. With the following *scheduler*, every five epochs, *lr* is multiplied by gamma = 0.1. \\
`scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)`

The *scheduler* needs access to the *lr* in the optimizer, so it is passed as an argument. The scheduler acts on *lr* only at the end of an epoch, and to make this action effective, you need to add this line outside the loader iteration loop:

`scheduler.step()`


**Exercise**: Integrate these elements into the training function. Check that the learning rate decreases as expected.


In [ ]:
def train_model_gpu_scheduler(model, loss_fn, optimizer,scheduler, num_epochs=1):
    # Record the starting time
    since = time.time()
    # Loop through epochs
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Iterate through training and validation phases
        for phase in ['train', 'val']:
            # Set the model to training mode during the training phase, and evaluation mode during validation
            if phase == 'train':
                model.train()
            else:
                model.eval()

            # Initialize counters for loss and correct predictions
            running_loss = 0.0
            running_corrects = 0

            # Iterate through batches in the data loader
            for inputs, labels in dataloaders[phase]:
                # Move inputs and labels to the specified device (same as in train_model_gpu())
                # YOUR CODE HERE
                raise NotImplementedError()

                # Zero the gradients in the optimizer (same as in train_model())
                # YOUR CODE HERE
                raise NotImplementedError()
                # Forward pass: compute model outputs and predictions (same as in train_model())
                with torch.set_grad_enabled(phase == 'train'):
                    # YOUR CODE HERE
                    raise NotImplementedError()
                    # Backward pass and optimization step if in the training phase (same as in train_model())
                    # YOUR CODE HERE
                    raise NotImplementedError()

                # Update counters
                running_loss += loss.item() * inputs.size(0)
                acc = torch.sum(preds == labels.data)
                # The 'acc' tensor is distributed across different parts of the GPU
                # Gather the 'acc' tensor on the CPU before accumulation (same as in train_model_gpu())
                # running_corrects += ...
                # YOUR CODE HERE
                raise NotImplementedError()

            # Calculate average loss and accuracy for the epoch
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            # Print epoch statistics
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {100*epoch_acc:.2f}%')
        # update the learning rate
        # print it (hint: look at optimizer.param_groups)
        # YOUR CODE HERE
        raise NotImplementedError()

    # Calculate and print the total training time
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    return model


In [ ]:
model = get_model(pretrained=False).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler =  torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

model = train_model_gpu_scheduler(model, loss_fn, optimizer, scheduler, num_epochs=10)

**Important note:** As you may notice, there are quite a few parameters to set, `learning rate`, `momentum`, `gamma`, `step_size`, etc... Typically, we use the parameterization that achieves the best scores on the validation dataset. To do this, we save the weights during training as soon as a validation record is reached. Note that by selecting the model this way, we are essentially training on the validation dataset. \\
This is why evaluating a selected model on the validation set always takes place on an independent **test dataset**, separate from the validation and training sets.

For reference, here's how to store model weights in PyTorch:
```python
import copy

# At the end of each epoch, save the best model:
if phase == "val" and epoch_acc > best_val_acc:
  best_model_wts = copy.deepcopy(model.state_dict())

# After training, before testing the model, load the best model:
model.load_state_dict(best_model_wts)
```

## **D.** Impact of pretraining on performance:

Training is faster on a GPU, but it only leads to a very poor score, barely better than random chance. To improve performance, a simple idea is to use a network trained on a similar (or more general) task as a starting point for learning. Here, it works particularly well with networks trained on ImageNet, whose convolutional filters are already very rich.

**Note:**
This method is refered to as **fine-tuning** a **pretrained model**.

**Exercise**:
Modify the training function to retrieve successive training and validation accuracies.

In [ ]:
def train(model, loss_fn, optimizer,scheduler, num_epochs=1):
    # Record the starting time
    since = time.time()
    # YOUR CODE HERE
    raise NotImplementedError()
    return model, accs


**Exercise:** Compare two ResNet18 trainings, one randomly initialized and the other pre-trained, using learning curves, over 25 epochs.

In [ ]:
max_epochs = 25
# Learning "from scratch" (random weights) :
# get the model
# Put the model on GPU
# get the loss, optimize, the scheduler and starting the training
# ...
# resnet_scratch, accs_scratch = train(...)
# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
# fine tuning a pretrained model:
# ...
# resnet_ft, accs_ft = train(...)
# YOUR CODE HERE
raise NotImplementedError()

**Learning curves**

In [ ]:
# Set the style
plt.style.use('seaborn-whitegrid')
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))
# Plot the data with improved style
epochs = np.arange(max_epochs)

# YOUR CODE HERE
raise NotImplementedError()
# Assuming accs_scratch and accs_ft are dictionaries with 'val' as a key
# you can use the following lines
#ax.plot(epochs, 100 * np.array(accs_scratch['val']), color='skyblue',
#        label='Training from Scratch', marker='o', linestyle='-', linewidth=2)
#ax.plot(epochs, 100 * np.array(accs_ft['val']), color='lightgreen',
#        label='Fine Tuning', marker='s', linestyle='-', linewidth=2)

# Set title and axis labels
ax.set_title('Validation Accuracy Over Epochs', fontsize=16)
ax.set_ylabel("Accuracy (%)", fontsize=14)
ax.set_xlabel("Epochs", fontsize=14)

# Set grid for better readability
ax.grid(True, linestyle='--', alpha=0.7)
# Add legend with a border and shadow
ax.legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9, fontsize=12)
plt.show()


The fine-tuning approach has many variations that fit into the broader framework of **transfer learning**. Partial fine-tuning, as illustrated in the following exercise, is one of these variations.

**Exercise:** Instead of retraining all the weights, you can simply use the weights of the classifier. This is referred to as *freezing* the other weights during retraining.
- Implement this approach and compare it with the previous ones.

In [ ]:
resnet = torchvision.models.resnet18(pretrained=True)

num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 2)

# freeze all the layers except the classifier (the last dense layers at end)
# YOUR CODE HERE
raise NotImplementedError()

freezed_resnet = resnet.to(device)
# train the model
# ...
# freezed_resnet, accs_freezing = train(...)
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
# Set the style
plt.style.use('seaborn-whitegrid')
# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))
# Plot the data with improved style
epochs = np.arange(max_epochs)

# YOUR CODE HERE
raise NotImplementedError()

# Set title and axis labels
ax.set_title('Validation Accuracy Over Epochs', fontsize=16)
ax.set_ylabel("Accuracy (%)", fontsize=14)
ax.set_xlabel("Epochs", fontsize=14)

# Set grid for better readability
ax.grid(True, linestyle='--', alpha=0.7)
# Add legend with a border and shadow
ax.legend(frameon=True, fancybox=True, shadow=True, framealpha=0.9, fontsize=12)
plt.show()


In the end, for this small dataset, retraining the last layer performs just as well as global training. To conclude, let's make some predictions with the model on the validation dataset:

In [ ]:
def visualize_model(model, num_images=10):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure(figsize=(25,num_images//5*5))

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//5, 5, images_so_far)
                ax.axis('off')
                imshow(inputs.cpu().data[j],ax,'Predicted: {}'.format(class_names[preds[j]]))

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(freezed_resnet)